In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler


### FewShotPromptTemplate ###
# 더 나은 대답을 얻기 위해 어떻게 대답해야 하는지 예제를 알려줌.
# prompt로 전달하는 것보다 더 좋은 결과를 얻을 수 있음.


examples = [
    {
        "country": "프랑스에 대해 무엇을 알고 있나요?",
        "answer": """
        제가 아는 정보는 다음과 같습니다:
        수도: 파리
        언어: 프랑스어
        음식: 와인과 치즈
        통화: 유로
        """,
    },
    {
        "country": "이탈리아에 대해 무엇을 알고 있나요?",
        "answer": """
        제가 알고 있는 내용입니다:
        수도: 로마
        언어: 이탈리아어
        음식: 피자와 파스타
        통화: 유로
        """,
    },
    {
        "country": "그리스에 대해 무엇을 알고 있나요?",
        "answer": """
        제가 알고 있는 내용입니다:
        수도: 아테네
        언어: 그리스어
        음식: 수블라키와 페타 치즈
        통화: 유로
        """,
    },
]


chat  = ChatOpenAI(model_name="gpt-4o-mini",
                temperature=0.1,
                streaming=True,
                callbacks=[StreamingStdOutCallbackHandler(),
                    ],
                )

# 각 예시가 어떤 형식으로 구성되어 있는지 알려줌
example_prompt = ChatPromptTemplate.from_messages([
        ("human","{country}에 대해 무엇을 알고 있나요?"),
        ("ai", "{answer}")
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 지리 전문가입니다. 짧은 대답을 하세요."),
    example_prompt,
    ("human", "{country}에 대해 무엇을 알고 있나요?"),
])

chain = final_prompt | chat

chain.invoke({
    "country":"mexico"
})